# Shooting Pattern Analysis

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import subprocess as sp
import pickle

import helper_basketball as h
import imp
imp.reload(h);

In [ ]:
## get all 2016-17 teams
params = {'LeagueID':'00','Season': '2016-17'}
teams = h.get_nba_data('commonTeamYears', params).set_index('TEAM_ID')
allteams = teams.loc[teams.MAX_YEAR=='2017'].index.values

## get all 2016-17 players
params = {'LeagueID':'00', 'Season': '2016-17', 'IsOnlyCurrentSeason': '0'}
players = h.get_nba_data('commonallplayers', params).set_index('PERSON_ID')
allplyrs = players.loc[players.TEAM_ID.isin(allteams)].index.values

In [ ]:
## params = {'PlayerID':'201939',
##           'PlayerPosition':'',
##           'Season':'2016-17',
##           'ContextMeasure':'FGA',
##           'DateFrom':'',
##           'DateTo':'',
##           'GameID':'',
##           'GameSegment':'',
##           'LastNGames':'0',
##           'LeagueID':'00',
##           'Location':'',
##           'Month':'0',
##           'OpponentTeamID':'0',
##           'Outcome':'',
##           'Period':'0',
##           'Position':'',
##           'RookieYear':'',
##           'SeasonSegment':'',
##           'SeasonType':'Regular Season',
##           'TeamID':'0',
##           'VsConference':'',
##           'VsDivision':''}
## 
## shotdf = []
## for p in allplyrs:
##     
##     ## get player p's data
##     params['PlayerID'] = p
##     shotdata = h.get_nba_data('shotchartdetail', params)
##     
##     ## subset columns
##     sc = shotdata.loc[:,'SHOT_DISTANCE':'SHOT_MADE_FLAG']
##     sc.drop('SHOT_ATTEMPTED_FLAG', axis=1)
##     
##     ## filter shots to 31 feet from end zone
##     sc = sc.loc[sc.LOC_Y < (31*12)]
##     
##     ## keep data with more than 50 shots
##     
##     if sc.SHOT_MADE_FLAG.sum() > 50:
##         
##         sc['PlayerID'] = p
##         shotdf += [sc]
## 
## allshots = pd.concat(shotdf)
## 
## pickle.dump(allshots, open('allshots2016-17.pkl', 'wb'))

In [ ]:
allshots = pickle.load(open('allshots2016-17.pkl', 'rb'))

In [ ]:
allmade = allshots.loc[allshots.SHOT_MADE_FLAG==1]
allmade.head()

In [ ]:
pd.DataFrame([allmade.LOC_X.describe(), 
              allmade.LOC_Y.describe()])

In [ ]:
## players info
player_ids = allmade.PlayerID.unique()
num_players = player_ids.size

## bin edges
xedges = (np.linspace(start=-25, stop=25, num=151, dtype=np.float)) * 12
yedges = (np.linspace(start= -4, stop=31, num=106, dtype=np.float)) * 12

nx = xedges.size - 1
ny = yedges.size - 1

## 2d histogram
all_counts = {}
all_smooth = {}

## data matrix: players (row) by vectorized 2-d court locations (column)
for i, one in enumerate(allmade.groupby('PlayerID')):
    
    pid, pdf = one
    
    tmp1, xedges, yedges = h.bin_shots(pdf, bin_edges=(xedges, yedges), density=True, sigma=2)
    tmp2, xedges, yedges = h.bin_shots(pdf, bin_edges=(xedges, yedges), density=False)
    
    all_smooth[pid] = tmp1.reshape(-1)
    all_counts[pid] = tmp2.reshape(-1)

In [ ]:
players.head()

In [ ]:
## select players from paper
stars = 'LeBron James|Brook Lopez|Tyson Chandler|Marc Gasol|Tony Parker|Kyrie Irving|Stephen Curry|James Harden|Steve Novak'
starids = players[players.DISPLAY_FIRST_LAST.str.contains(stars)].loc[player_ids].dropna()

In [ ]:
fig, ax = plt.subplots(starids.shape[0], 2, figsize=(20,60))

for axi, plyri in enumerate(starids.index.values):
    h.plot_shotchart(all_counts[plyri], xedges, yedges, ax=ax[axi,0])
    h.plot_shotchart(all_smooth[plyri], xedges, yedges, ax=ax[axi,1])
    ax[axi,0].set_title(players.DISPLAY_FIRST_LAST[plyri])
    ax[axi,1].set_title(players.DISPLAY_FIRST_LAST[plyri])

In [ ]:
## Non-negative Matrix Factorization

import sklearn.decomposition as skld

X = np.stack(all_smooth.values())

model = skld.NMF(n_components=10, init='nndsvda', max_iter=500, random_state=0)
W = model.fit_transform(X.T)
H = model.components_

In [ ]:
fig, ax = plt.subplots(5, 2, figsize=(20,40))

for i, axi in enumerate(ax.flatten()):
    h.plot_shotchart(W[:,i], xedges, yedges, ax=axi)
    axi.set_title('NMF component ' + str(i))